In [38]:
import json
import pandas as pd

In [34]:
path = 'C:\\Users\\tnv664\\OneDrive - University of Copenhagen\\Documents\\Uni\\Ph.D\\deep learning\\02456-deep-learning-with-PyTorch\\project - shared data folde\\billsum_v4_1\\billsum_v4_1\\'
path = 'C:\\Users\\tnv664\\OneDrive - University of Copenhagen\\Documents\\Uni\Ph.D\\deep learning\\02456-deep-learning-with-PyTorch\\project - shared data folde\\billsum_v4_1\\billsum_v3\\'

In [35]:
with open(path +'us_train_data_final_OFFICIAL.jsonl', 'r') as json_file:
    json_list = list(json_file)


In [36]:
len(json_list)

18949

In [37]:
json.loads(json_list[-1]).keys()

dict_keys(['text', 'summary', 'bill_id', 'title', 'text_len', 'sum_len'])

In [43]:
df = pd.read_json(path +'us_train_data_final_OFFICIAL.jsonl', lines=True)

In [44]:
df.bill_id.sort_values()

4285     103_hr1002
14581    103_hr1003
17535    103_hr1005
9167     103_hr1007
2940     103_hr1013
            ...    
2478       115_s972
1324        115_s98
1076       115_s982
11587      115_s992
2794       115_s995
Name: bill_id, Length: 18949, dtype: object

In [46]:
#import jsonlines
import os
import pandas as pd
import pickle
import re

def replace_semicolon(text, threshold=10):
    '''
    Get rid of semicolons.
    First split text into fragments between the semicolons. If the fragment 
    is longer than the threshold, turn the semicolon into a period. O.w treat
    it as a comma.
    Returns new text
    '''
    new_text = ""
    for subset in re.split(';', text):
        subset = subset.strip() # Clear off spaces
        # Check word count
        if len(subset.split()) > threshold:
            # Turn first char into uppercase
            new_text += ". " + subset[0].upper() + subset[1:]
        else:
            # Just append with a comma 
            new_text += ", " + subset

    return new_text

USC_re = re.compile('[Uu]\.*[Ss]\.*[Cc]\.]+')
PAREN_re = re.compile('\([^(]+\ [^\(]+\)')
BAD_PUNCT_RE = re.compile(r'([%s])' % re.escape('"#%&\*\+/<=>@[\]^{|}~_'), re.UNICODE)
BULLET_RE = re.compile('\n[\ \t]*`*\([a-zA-Z0-9]*\)')
DASH_RE = re.compile('--+')
WHITESPACE_RE = re.compile('\s+')
EMPTY_SENT_RE = re.compile('[,\.]\ *[\.,]')
FIX_START_RE = re.compile('^[^A-Za-z]*')
FIX_PERIOD = re.compile('\.([A-Za-z])')
SECTION_HEADER_RE = re.compile('SECTION [0-9]{1,2}\.|\nSEC\.* [0-9]{1,2}\.|Sec\.* [0-9]{1,2}\.')

FIX_PERIOD = re.compile('\.([A-Za-z])')

SECTION_HEADER_RE = re.compile('SECTION [0-9]{1,2}\.|\nSEC\.* [0-9]{1,2}\.|Sec\.* [0-9]{1,2}\.')

def clean_text(text):
    """
    Borrowed from the FNDS text processing with additional logic added in.
    Note: we do not take care of token breaking - assume SPACY's tokenizer
    will handle this for us.
    """

    # Indicate section headers, we need them for features
    text = SECTION_HEADER_RE.sub('SECTION-HEADER', text)
    # For simplicity later, remove '.' from most common acronym
    text = text.replace("U.S.", "US")
    text = text.replace('SEC.', 'Section')
    text = text.replace('Sec.', 'Section')
    text = USC_re.sub('USC', text)

    # Remove parantheticals because they are almost always references to laws 
    # We could add a special tag, but we just remove for now
    # Note we dont get rid of nested parens because that is a complex re
    #text = PAREN_re.sub('LAWREF', text)
    text = PAREN_re.sub('', text)
    

    # Get rid of enums as bullets or ` as bullets
    text = BULLET_RE.sub(' ',text)
    
    # Clean html 
    text = text.replace('&lt;all&gt;', '')

    # Remove annoying punctuation, that's not relevant
    text = BAD_PUNCT_RE.sub('', text)

    # Get rid of long sequences of dashes - these are formating
    text = DASH_RE.sub( ' ', text)

    # removing newlines, tabs, and extra spaces.
    text = WHITESPACE_RE.sub(' ', text)
    
    # If we ended up with "empty" sentences - get rid of them.
    text = EMPTY_SENT_RE.sub('.', text)
    
    # Attempt to create sentences from bullets 
    text = replace_semicolon(text)
    
    # Fix weird period issues + start of text weirdness
    #text = re.sub('\.(?=[A-Z])', '  . ', text)
    # Get rid of anything thats not a word from the start of the text
    text = FIX_START_RE.sub( '', text)
    # Sometimes periods get formatted weird, make sure there is a space between periods and start of sent   
    text = FIX_PERIOD.sub(". \g<1>", text)

    # Fix quotes
    text = text.replace('``', '"')
    text = text.replace('\'\'', '"')

    # Add special punct back in
    text = text.replace('SECTION-HEADER', '<SECTION-HEADER>')

    return text




In [53]:
df['clean_text'] = df['text'].apply(clean_text)

In [61]:
df.clean_text[9]

'<SECTION-HEADER> SHORT TITLE. This Act may be cited as the "Indian Needs Assessment and Program Evaluation Act of 2001". <SECTION-HEADER> FINDINGS, PURPOSES. Findings. Congress finds that the United States and the Indian tribes have a unique legal and political government-to-government relationship. Pursuant to the Constitution, treaties, statutes, Executive orders, court decisions, and course of conduct, the United States has a trust obligation to provide certain services to Indian tribes and to Indians. Federal departments and agencies charged with administering programs and providing services to, or for the benefit of, Indians have not furnished Congress with adequate information necessary to assess such programs on the needs of Indians and Indian tribes. Such lack of information has hampered the ability of Congress to determine the nature, type, and magnitude of such needs as well as its ability to respond to them. And Congress cannot properly fulfill its obligation to Indian trib

In [47]:

#if __name__ == '__main__':


 #   prefix = os.environ['BILLSUM_PREFIX']
  #  path = os.path.join(prefix, 'data_final')

    # Create dir where data should be saved
   # save_path = os.path.join(prefix, 'clean_final')
    #os.mkdir(save_path)

for i, file in enumerate(df): #os.listdir(path):
    #if '.jsonl' not in file:
        #continue


    #file_path = os.path.join(path,  file)

    #print("Now processing", file_path)

    #data = pd.read_json(file_path, lines=True)
    data = df.iloc
    
    data['clean_text'] = data.text.map(clean_text)

    data['clean_summary'] = data.summary.map(clean_text)

    data['clean_title'] = data.title.map(clean_text)

   # save_path = os.path.join(prefix, 'clean_final', file)

   # data.to_json(save_path, lines=True, orient='records')


KeyError: 'BILLSUM_PREFIX'